In [8]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool, DirectoryReadTool, FileReadTool
from crewai.tools import BaseTool

In [2]:
local_llm = LLM(
    model="ollama/llama3.1",
    base_url="http://localhost:11434"
)

cloud_llm = LLM(
    model="gemini-2.5-flash",
    api_key=os.getenv("GEMINI_API_KEY")
)

In [ ]:
# Agents

sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match our ideal customer profile",
    backstory="""
        As a part of the dynamic sales team at CrewAI, your mission is to scour
        the digital landscape for potential leads. Armed with cutting-edge tools
        and a strategic mindset, you analyze data, trends, and interactions to
        unearth opportunities that others might overlook.Your work is crucial in 
        paving the way for meaningful engagements and driving the company's growth."
    """,
    allow_delegation=False,
    llm=cloud_llm,
    verbose=True
)

lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory="""
        Within the vibrant ecosystem of CrewAI's sales department,
        you stand out as the bridge between potential clients
        and the solutions they need.
        By creating engaging, personalized messages,
        you not only inform leads about our offerings
        but also make them feel seen and heard.
        Your role is pivotal in converting interest
        into action, guiding leads through the journey
        from curiosity to commitment.
    """,
    allow_delegation=False,
    llm=cloud_llm,
    verbose=True
)
    

In [6]:
# Instantiate the tools

directory_read_tool = DirectoryReadTool(directory="./instructions")
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [9]:
# Custom tool

class SentimentAnalysisTool(BaseTool):
    name: str = "sentiment_analysis_tool"
    description: str = "Analyses the sentiment of text to ensure positive and engaging communication."

    def _run(self, text: str) -> str:
        # Implement sentiment analysis logic here
        # For example, using a pre-trained model or library
        return "Positive" # For now just return a hardcoded value to test custom tool

In [11]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [12]:
# Tasks

lead_profiling_task = Task(
    description="""
    Conduct an in-depth analysis of {lead_name}, a company in the {industry}
    sector that recently showed interest in our solutions. Utilize all available
    data sources to compile a detailed profile, focusing on key decision-makers,
    recent business developments, and potential needs that align with our
    offerings. This task is crucial for tailoring our engagement strategy
    effectively.
    Don't make assumptions and only use information you absolutely sure about.
    """,
    expected_output="""
    A comprehensive report on {lead_name}, including company background,
    key personnel, recent milestones, and identified needs. Highlight
    potential areas where our solutions can provide value, and suggest
    personalized engagement strategies.
    """,
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

personalized_outreach_task = Task(
    description="""
        Using the insights gathered from the lead profiling report on {lead_name},
        craft a personalized outreach campaign aimed at {key_decision_maker},
        the {position} of {lead_name}. The campaign should address their recent
        {milestone} and how our solutions can support their goals. Your
        communication must resonate with {lead_name}'s company culture and values,
        demonstrating a deep understanding of their business and needs.
        Don't make assumptions and only use information you absolutely sure about.
        """,
    expected_output="""
        A series of personalized email drafts tailored to {lead_name}, specifically
        targeting {key_decision_maker}. Each draft should include a compelling
        narrative that connects our solutions with their recent achievements and
        future goals. Ensure the tone is engaging, professional, and aligned with
        {lead_name}'s corporate identity.
        """,
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [ ]:
# Crew

crew = Crew(
    agents=[sales_rep_agent, lead_sales_rep_agent],
    tools=[directory_read_tool, file_read_tool, search_tool],
    verbose=True,
    memory=True
)